In [1]:
#search for change in dice loss for single sample
import pandas as pd
import numpy as np
import os
from monai.metrics import compute_dice
import glob
import torch


/home/vscode/.local/lib/python3.10/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [16]:
dvolumeT = pd.read_csv('/workspaces/data/MegaGen/logs/SCORE/CSVS/id_test_volume.csv')
dvolTS = dvolumeT[dvolumeT['volume'] <3000]
dvolTS['id'] = dvolTS['id'].apply(lambda x: f'{int(x):04d}')

/tmp/ipykernel_2077/821571249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dvolTS['id'] = dvolTS['id'].apply(lambda x: f'{int(x):04d}')


In [ ]:
root = '/workspaces/data/brain_meningioma'
def getScoreS(df_slice):
    dScore = pd.DataFrame() # columns=['maskTfile', 'dscore'])
    for id in df_slice['id']:
        maskTfile = os.path.join('npy', f'BraTS-MEN-RT-{id}-1_slice_*_mask.npy')
        # probTfile = os.path.join('pt', f'BraTS-MEN-RT-{id}-1_slice_*_mask.pt')
        maskFiles = glob.glob(os.path.join(root, 'slice/s_test', maskTfile))
        for maskFile in maskFiles:
            probFile0 = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
            probFile0 = os.path.join(root, 'oProb/unet1s/s_test', probFile0)
            y_pred0 = torch.load(probFile0,weights_only=False).unsqueeze(0).unsqueeze(0)>0.5
            
            probFile1 = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
            probFile1 = os.path.join(root, 'oProb/unetpp0D/s_test', probFile1)
            y_pred1 = torch.load(probFile1,weights_only=False).unsqueeze(0).unsqueeze(0)>0.5
            
            y = torch.from_numpy(np.load(maskFile)).unsqueeze(0).unsqueeze(0).cuda(0)
            d0= compute_dice(y_pred0, y).item()
            d1= compute_dice(y_pred1, y).item()
            dScore = pd.concat([dScore, pd.DataFrame
                                ([[os.path.relpath(maskFile,os.path.join(root,'slice/s_test')), d0, d1]], 
                                    columns=['maskTfile', 'd0','d1'])])
    return dScore
        
    # probIFiles = glob.glob(f'/workspaces/data/brain_meningioma/oProb/unet1s/s_test/pt/BraTS-MEN-RT-{id}-2_slice_*.pt')


In [38]:
maskTfile

'npy/BraTS-MEN-RT-0109-1_slice_*_mask.npy'

In [ ]:
dScore = getScoreS(dvolTS)
dScore['dadd'] = dScore['d1'] - dScore['d0']

In [49]:
dScore.sort_values(by='dadd', ascending=False).head(20)

,maskTfile,d0,d1,dadd
0,npy/BraTS-MEN-RT-0264-1_slice_062_mask.npy,0.000000,0.800000,0.800000
0,npy/BraTS-MEN-RT-0053-1_slice_124_mask.npy,0.000000,0.742857,0.742857
0,npy/BraTS-MEN-RT-0165-1_slice_068_mask.npy,0.000000,0.742358,0.742358
0,npy/BraTS-MEN-RT-0264-1_slice_063_mask.npy,0.102041,0.772727,0.670686
0,npy/BraTS-MEN-RT-0165-1_slice_067_mask.npy,0.190045,0.853755,0.663710
0,npy/BraTS-MEN-RT-0053-1_slice_123_mask.npy,0.299065,0.862069,0.563004
0,npy/BraTS-MEN-RT-0264-1_slice_064_mask.npy,0.000000,0.548387,0.548387
0,npy/BraTS-MEN-RT-0053-1_slice_128_mask.npy,0.343750,0.881356,0.537606
0,npy/BraTS-MEN-RT-0092-1_slice_066_mask.npy,0.000000,0.535211,0.535211
0,npy/BraTS-MEN-RT-0320-1_slice_168_mask.npy,0.382114,0.850394,0.468280


In [ ]:
dLobe = pd.read_csv('/workspaces/data/MegaGen/logs/SCORE/CSVS/id_test_lobe.csv',index_col=0)
dLobeT = dLobe[dLobe['lobe_overlap'] == 'Temporal']
dLobeT['id'] = dLobeT['id'].apply(lambda x: f'{int(x):04d}')
dScoreLT = getScoreS(dLobeT)
dScoreLT['dadd'] = dScoreLT['d1'] - dScoreLT['d0']

/tmp/ipykernel_2077/1968581021.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dLobeT['id'] = dLobeT['id'].apply(lambda x: f'{int(x):04d}')


TypeError: 'NoneType' object is not subscriptable

In [ ]:
dLo

,id,lobe_overlap
8,0086,Temporal
10,0092,Temporal
11,0109,Temporal
15,0165,Temporal
21,0238,Temporal
25,0264,Temporal
30,0323,Temporal
32,0349,Temporal
38,0416,Temporal
46,0577,Temporal


In [ ]:
#trying ID-dice
root = '/workspaces/data/brain_meningioma'
dScore = pd.DataFrame() # columns=['maskTfile', 'dscore'])
for id in dvolTS['id']:
    maskTfile = os.path.join('npy', f'BraTS-MEN-RT-{id}-1_slice_*_mask.npy')
    # probTfile = os.path.join('pt', f'BraTS-MEN-RT-{id}-1_slice_*_mask.pt')
    maskFiles = glob.glob(os.path.join(root, 'slice/s_test', maskTfile))
    for maskFile in maskFiles:
        y_preds = []
        for model in ['unet1s', 'unetpp0', 'unetpp0D']:
            probFile = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
            probFile = os.path.join(root, f'oProb/{model}/s_test', probFile)
            y_pred = torch.load(probFile,weights_only=False).unsqueeze(0).unsqueeze(0)>0.5
            y_preds.append(y_pred)
        
        y = torch.from_numpy(np.load(maskFile)).unsqueeze(0).unsqueeze(0).cuda(0)
        dices = [compute_dice(y_pred, y).item() for y_pred in y_preds]
        dScore = pd.concat([dScore, pd.DataFrame
                            ([[os.path.relpath(maskFile,os.path.join(root,'slice/s_test')), dices]], 
                                columns=['maskTfile', 'dices'])])
    
    # probIFiles = glob.glob(f'/workspaces/data/brain_meningioma/oProb/unet1s/s_test/pt/BraTS-MEN-RT-{id}-2_slice_*.pt')

In [ ]:
dScore['dadd'] = dScore['dices'].apply(lambda x:x[2])-dScore['dices'].apply(lambda x:x[0])

In [71]:
dScore.sort_values(by='dadd', ascending=False).head(10).to_numpy()

array([['npy/BraTS-MEN-RT-0264-1_slice_062_mask.npy',
        list([0.0, 0.4126984179019928, 0.800000011920929]),
        0.800000011920929],
       ['npy/BraTS-MEN-RT-0053-1_slice_124_mask.npy',
        list([0.0, 0.42553192377090454, 0.7428571581840515]),
        0.7428571581840515],
       ['npy/BraTS-MEN-RT-0165-1_slice_068_mask.npy',
        list([0.0, 0.5793103575706482, 0.7423580884933472]),
        0.7423580884933472],
       ['npy/BraTS-MEN-RT-0264-1_slice_063_mask.npy',
        list([0.10204081982374191, 0.7586206793785095, 0.7727272510528564]),
        0.6706864312291145],
       ['npy/BraTS-MEN-RT-0165-1_slice_067_mask.npy',
        list([0.19004525244235992, 0.6805970072746277, 0.8537549376487732]),
        0.6637096852064133],
       ['npy/BraTS-MEN-RT-0053-1_slice_123_mask.npy',
        list([0.29906541109085083, 0.5316455960273743, 0.8620689511299133]),
        0.5630035400390625],
       ['npy/BraTS-MEN-RT-0264-1_slice_064_mask.npy',
        list([0.0, 0.58823531866073

In [ ]:
#3D dice

root = '/workspaces/data/brain_meningioma'
dScore = pd.DataFrame() # columns=['maskTfile', 'dscore'])
for id in dvolTS['id']:
    maskTfile = os.path.join('npy', f'BraTS-MEN-RT-{id}-1_slice_*_mask.npy')
    # probTfile = os.path.join('pt', f'BraTS-MEN-RT-{id}-1_slice_*_mask.pt')
    maskFiles = sorted(glob.glob(os.path.join(root, 'slice/s_test', maskTfile)))
    for model in ['unet1s', 'unetpp0', 'unetpp0D']:
        y_predIDSs = []
        for maskFile in maskFiles:
            probFile = os.path.relpath(maskFile, os.path.join(root, 'slice/test')).replace('npy', 'pt')
            probFile = os.path.join(root, f'oProb/{model}/s_test', probFile)
            y_pred = torch.load(probFile,weights_only=False)>0.5
            print('y_pred shape: ', y_pred.shape)
            y_predIDSs.append(y_pred)
            #torch.stack(y_predID)
            #rearrange(y_predID, )

        # y = torch.from_numpy(np.load(maskFile)).unsqueeze(0).unsqueeze(0).cuda(0)
        # dices = [compute_dice(y_pred, y).item() for y_pred in y_preds]
        # dScore = pd.concat([dScore, pd.DataFrame
        #                     ([[os.path.relpath(maskFile,os.path.join(root,'slice/s_test')), dices]], 
        #                         columns=['maskTfile', 'dices'])])
    

y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:  torch.Size([1, 1, 1, 160, 192])
y_pred shape:

KeyboardInterrupt: 